In [3]:
from Base_data import *
import yfinance as yf

class Screaner_Tickers(Base_Data):

    def run_tsi_strategy(self, first_smo, second_smo, tsi_smo):
        ''' Backtesting a SMA-based strategy.

        Parameters
        ==========
        first_smo, second_smo, tsi_smo: int
            
        '''
        msg = f'\n\nRunning TSI STRATEGY | Slow={first_smo} & Fast={second_smo}  & Sign={tsi_smo}'
        
        #armado de los indicadores de TSI
        self.data["PC"] =self.data["Close"].diff()
        self.data["PCABS"] = abs(self.data["PC"])
        self.data["PCABS"] = abs(self.data["PC"])
        self.data['TSI_First_Smo'] =self.data["PCABS"].ewm(span=first_smo).mean()
        self.data['TSI_Second_Smo'] =self.data['TSI_First_Smo'].ewm(span=second_smo).mean()
        self.data['TSI_First_Smo_PC'] =self.data["PC"].ewm(span=first_smo).mean()
        self.data['TSI_Second_Smo_PC'] =self.data['TSI_First_Smo_PC'].ewm(span=second_smo).mean()
        #aqui TSI e TSI SmoSMA son los dos indicadores a utilizar 
        self.data["TSI"] = 100*(self.data['TSI_Second_Smo_PC']/self.data['TSI_Second_Smo']) 
        self.data["TSI_Smo_SMA"] =self.data["TSI"].ewm(span=tsi_smo).mean()

        #se elimina los Nan
        self.data = self.data.copy().dropna()
        #se pone valor 1 encaso que la señal sea compradora y -1 en caso que sea vendedora
        self.data['position'] = np.where(self.data["TSI"] > self.data["TSI_Smo_SMA"], 1, -1)
        data = pd.DataFrame(self.data['position'])

        #se toma el ultimo dato, para saber si la señal debe ser compradora o vendedora
        if self.data['position'].iloc[-1] == 1:
            self.place_buy_order
        elif self.data['position'].iloc[-1] == -1:
            self.place_sell_order



        #return print(f'{self.symbol}\n  {data}')    

if __name__ == '__main__':
    tickers = ['MELI']
    for ticker in tickers:
        lobt = Screaner_Tickers(f'{ticker}','2021-10-01','2022-03-10',verbose=False)
        lobt.run_tsi_strategy(25,13,13)

[*********************100%***********************]  1 of 1 completed


TypeError: 'Screaner_Tickers' object is not subscriptable